$$
\begin{array}{c}
\LARGE \textbf{Trabajo\ Final\ de\ Comunicaciones\ Digitales}
\end{array}
$$

$$
\begin{aligned}
&\textbf{Marco Teórico} \\[1em]
&\text{El auge del Internet de las Cosas (IoT) ha impulsado el desarrollo de tecnologías} \\
&\text{de comunicación de bajo consumo y largo alcance, conocidas como Low Power Wide} \\
&\text{Area Networks (LPWAN). Dentro de este marco, LoRa (Long Range) es el estándar} \\
&\text{que se ha consolidado como una tecnología ideal para la comunicación inalámbrica} \\
&\text{de largo alcance y bajo consumo energético.} \\[1em]
&\text{En este sentido, el artículo propuesto por la cátedra introduce una formulación} \\
&\text{matemática para la modulación LoRa, la cual puede describirse como una modulación} \\
&\text{chirp con desplazamiento de frecuencia (Frequency Shift Chirp Modulation, FSCM).} \\
&\text{Esta técnica se basa en la codificación de bits en símbolos enteros y en la generación} \\
&\text{de señales chirp cuya frecuencia depende de dichos símbolos. Además, en esta formulación,} \\
&\text{la información está codificada en el desplazamiento inicial de frecuencia del chirp,} \\
&\text{y no depende de la pendiente ni en la duración de la señal.} \\[1em]
&\text{Este enfoque permite definir una base ortogonal de señales discretas, lo que favorece} \\
&\text{una demodulación eficiente mediante la Transformada Rápida de Fourier (FFT).}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Desarrollo} \\[1em]
&\text{Este proceso de codificación, tal como se describe matemáticamente en el artículo, puede} \\
&\text{descomponerse en tres etapas fundamentales. La primera de ellas es el mapeo de bits a un} \\
&\text{símbolo decimal, y comienza con un vector $w(nT_s)$ compuesto por una cantidad fija de bits} \\
&\text{determinada por el parámetro Spreading Factor (SF). Este factor representa la cantidad de } \\
&\text{ bits por símbolo y se define como:} \\[1em]
&\hspace{2cm} SF = \log_2(M) \\[1em]
&\text{Donde $M$ es la cantidad total de símbolos posibles (cardinalidad de la modulación). Por ejemplo,} \\
&\text{si $SF = 5$, existen $2^5 = 32$ símbolos distintos.} \\
&\text{El valor decimal resultante de este mapeo, denotado como $s(nT_s)$, se obtiene al sumar los valores} \\
&\text{ponderados de los bits del vector $w(nT_s)$. Este número representa directamente la frecuencia inicial} \\
&\text{desde la cual se genera la señal chirp para su transmisión.} \\[1em]
&\textbf{Fórmula de codificación:} \\[1em]
&\hspace{2cm} s(nT_s) = \sum_{h=0}^{SF-1} w_h(nT_s) \cdot 2^h \\[1em]
&\text{donde:} \\
&\hspace{1cm} ~ s(nT_s) \in \{0, ..., 2^{SF} - 1\} \text{ es el símbolo codificado} \\
&\hspace{1cm} ~ w_h(nT_s) \in \{0,1\} \text{ son los bits del bloque actual, ordenados desde el LSB} \\
&\hspace{1cm} ~ SF \text{ es el Spreading Factor (cantidad de bits por símbolo)} \\
&\hspace{1cm} ~ h \text{ es el índice del bit dentro del bloque} \\[1em]
&\textbf{Parámetros de transmisión:} \\[1em]
&\text{Supongamos que el ancho de banda del canal utilizado es $B$, lo que implica que transmitimos una muestra cada:} \\
&\hspace{2cm} T = \frac{1}{B} \\[1em]
&\text{Un símbolo $s(nT_s)$ se transmite cada:} \\
&\hspace{2cm} T_s = 2^{SF} \cdot T \\[1em]
&\text{Cada símbolo representa un número entero entre $0$ y $2^{SF} - 1$. Esto no solo determina la frecuencia} \\
&\text{con la que inicia el chirp, sino también cuán "extendida" estará la señal en el tiempo. A mayor $SF$,} \\
&\text{mayor será la duración del símbolo y mayor la resistencia al ruido, a costa de menor tasa de transmisión.} \\[1em]
&\text{Por otro lado, el decodificador es el componente encargado de inferir el mensaje original a partir de la señal recibida.} \\
&\text{En un canal aditivo de ruido blanco gaussiano (AWGN), la estrategia óptima es la de Máxima Verosimilitud (ML).} \\
&\text{Esta consiste en proyectar la señal recibida sobre todas las señales base posibles y elegir aquella cuya proyección} \\
&\text{maximice la expresión:} \\[1em]
&\hspace{2cm} \max_{c} \left| \langle y, c \rangle \right| \\[1em]
&\text{Donde $c$ es una señal candidata y $y$ es la señal observada. Gracias a la ortogonalidad de las señales en FSCM,} \\
&\text{esta proyección puede implementarse eficientemente con una FFT.}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Implementación en Python}
\end{aligned}
$$

In [ ]:
import numpy as np

def bits_to_symbols(bit_array, SF):
    """Agrupa los bits en bloques de SF y los convierte en símbolos enteros."""
    bit_array = np.array(bit_array).reshape(-1, SF)
    powers = 2 ** np.arange(SF)[::-1]
    symbols = bit_array.dot(powers)
    return symbols

In [ ]:
def symbols_to_bits(symbols, SF):
    """Convierte los símbolos de vuelta a su representación binaria."""
    bits = ((symbols[:, None] & (1 << np.arange(SF)[::-1])) > 0).astype(int)
    return bits.reshape(-1)

In [ ]:
def simulate_encoder_decoder(SF, total_bits):
    """Genera bits aleatorios, codifica, decodifica y calcula el BER."""
    assert total_bits % SF == 0, "El número total de bits debe ser múltiplo de SF"

    # Generar bits aleatorios con distribución uniforme
    tx_bits = np.random.randint(0, 2, total_bits)

    # Codificación
    tx_symbols = bits_to_symbols(tx_bits, SF)

    # Transmisión simulada perfecta (sin ruido)
    rx_symbols = tx_symbols.copy()  # En canal real, se podría agregar ruido o errores

    # Decodificación
    rx_bits = symbols_to_bits(rx_symbols, SF)

    # Cálculo de BER
    bit_errors = np.sum(tx_bits != rx_bits)
    ber = bit_errors / total_bits

    # Resultados
    print("Bits transmitidos (primeros 64):  ", tx_bits[:64])
    print("Bits decodificados (primeros 64):", rx_bits[:64])
    print(f"\nTotal de bits transmitidos: {total_bits}")
    print(f"Errores totales: {bit_errors}")
    print(f"BER (Bit Error Rate): {ber:.6f}")

# Parámetros de simulación
SF = 7  # Spreading Factor
num_bits = SF * 1000  # Enviar 1000 símbolos

simulate_encoder_decoder(SF, num_bits)

## Ecuación 2 y 3: Modulación Chirp 

**Ecuación 2 (forma general):**

$$c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi[(s(nT_s)+k) \bmod 2^{SF}]\frac{kT}{B}} \quad (2)$$

donde:
- $c(nT_s + kT)$ es la señal chirp modulada en el tiempo $nT_s + kT$
- $\frac{1}{\sqrt{2^{SF}}}$ es el factor de normalización de energía (asegura energía unitaria para cada símbolo)
- $s(nT_s)$ es el símbolo codificado (de la Ecuación 1)
- $k$ es el índice de muestra dentro del símbolo ($k = 0, 1, ..., 2^{SF}-1$)
- $T$ es el período de muestreo
- $B$ es el ancho de banda del canal
- $\bmod 2^{SF}$ indica la operación módulo $2^{SF}$

### Características principales:

1. **Chirp complejo**: La señal tiene frecuencia linealmente creciente dentro de cada símbolo
2. **Frecuencia instantánea**: Depende de $(s(nT_s)+k) \bmod 2^{SF}$, que representa un corrimiento de frecuencia específico para cada símbolo
3. **Ortogonalidad**: Los diferentes símbolos generan chirps ortogonales entre sí, facilitando la decodificación
4. **Normalización**: El factor $\frac{1}{\sqrt{2^{SF}}}$ mantiene la energía constante por símbolo


  
**Ecuación 3 (forma simplifca:**)


Dado que $T = \frac{1}{B}$, se reemplaza por $T \cdot B = 1$, simplificando la ecuación:

$$c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi \cdot \frac{[(s(nT_s)+k) \bmod 2^{SF}] \cdot k}{2^{SF}}} \quad (3)$$

**Pasos** 

1. Definiciíon del tiempo discreto:  

Teniendo en cuento lo definido más arriba: 
 
$$t = nT_s + kT = n \cdot 2^ SF \cdot T + kT$$

2. Codificar la frecuencia en el chirp: 

La modulacíon LoRa usa un chirp de frecuencia creciente, donde la frecuencia instantánea varía linealmente con k, y 
se desplaza según el valor del símbolo $$s(nT_s)$$ 
$$\frac{[(s+k) \bmod 2^{SF}] \cdot k}{2^{SF}}$$


3. Fase acumulada y señal compleja 

$$e^{j2\pi \cdot \frac{[(s+k) \bmod 2^{SF}] \cdot k}{2^{SF}}}$$

Representa una señal compleja cuya fase cambia con el tiempo (índice k).Esa fase varía en funcíon del símbolo transmitido s y de la muestra actual k. 
Como resultado, se genera una onda chirp (señal cuya frecuencia aumenta linealmente con k ). Esto se logra porque la fase del exponente crece de forma cuadrática con k, algo característico de un chirp lineal. 
El desplazamiento s hace que cada símbolo comience su chirp en una frecuencia distinta, lo que permite codificar la información.

4. Normalizacíon de energía 

El factor $\frac{1}{\sqrt{2^{SF}}}$ normaliza la señal para que tenga energía unitaria. 

$\text{Implementación en python}$

<text>La función bits_to_symbols: Agrupa un arreglo plano de bits en bloques de longitud igual al Spreading Factor (SF) y 

convierte cada bloque en un número decimal. Este paso simula lo que en el paper se describe como la generación del símbolo
 
𝑠(𝑛𝑇𝑠) a partir del vector de bits 𝑤(𝑛𝑇𝑠)

In [ ]:
import random
import numpy as np

def bits_to_symbols(bit_array, SF):
    """Agrupa los bits